# Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(Note that types can not be redefined - you have to restart your Julia session to change a type definiton.)

In [ ]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/latest/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [ ]:
methods(MyType)

In [ ]:
m = MyType()

In [ ]:
typeof(m)

In [ ]:
m isa MyType

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [ ]:
struct A
    x::Int64
end

In [ ]:
A()

The default constructor always expects values for all fields.

In [ ]:
A(3)

In [ ]:
a = A(3)

In [ ]:
# a.<TAB>
a.x

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [ ]:
a.x = 2

In [ ]:
mutable struct B
    x::Int64
end

In [ ]:
b = B(3)

In [ ]:
b.x

In [ ]:
b.x = 4

In [ ]:
b.x

Note, however, that **immutability is not recursive**.

In [ ]:
struct C
    x::Vector{Int64}
end

In [ ]:
c = C([1, 2, 3])

In [ ]:
c.x

In [ ]:
c.x = [3,4,5]

In [ ]:
c.x[1] = 3

In [ ]:
c.x

In [ ]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

Abstract types are just as easy to define using the keyword `abstract type`.

In [ ]:
abstract type MyAbstractType end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [ ]:
struct MyConcreteType <: MyAbstractType # subtype
    somefield::String
end

In [ ]:
c = MyConcreteType("test")

In [ ]:
c isa MyAbstractType

In [ ]:
supertype(MyConcreteType)

In [ ]:
subtypes(MyAbstractType)

# Custom constructor

In [ ]:
struct VolNaive
    value::Float64
end

In [ ]:
VolNaive(3.0)

In [ ]:
VolNaive(-3.0)

In [ ]:
struct VolSimple
    value::Float64
    
    function VolSimple(x) # inner constructor. function name must match the type name.
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new(x) # within an inner constructor, the `new` function can be used to create an object.
    end
end

---

**Side note:**

```julia
if !(x isa Real)
    throw(ArgumentError("Must be real"))
end
if x < 0
    throw(ArgumentError("Negative volume not allowed."))
end
```

This can be written more compactly as
```julia
x isa Real || throw(ArgumentError("Must be real"))
x < 0 && throw(ArgumentError("Negative volume not allowed."))
```

See ["short-circuit evaluation"](https://docs.julialang.org/en/latest/manual/control-flow/#Short-Circuit-Evaluation-1) for more information.

---

In [ ]:
VolSimple(3.0)

In [ ]:
VolSimple(-3.0)

In [ ]:
VolSimple("test")

In [ ]:
VolSimple(3) # implicit conversion from Int64 -> Float64

# Parametric types

Volumes don't have to be `Float64` values. We can easily relax our type definition to allow all sorts of internal value types.

In [ ]:
struct VolParam{T}
    value::T
    
    function VolParam(x::T) where T # x can be of any type T
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new{T}(x) # Note that we need an extra {T} here
    end
end

In [ ]:
VolParam(3.0)

In [ ]:
VolParam(3)

Instead of checking the realness of the input `x` explicitly in the inner constructor, we can impose type constraints in the type and function signatures.

In [ ]:
struct Vol{T<:Real} <: Real # the last <: Real tells Julia that a Vol is a subtype of Real, i.e. basically a real number
    value::T
    
    function Vol(x::T) where T<:Real # x can be of any type T<:Real
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new{T}(x)
    end
end

In [ ]:
Vol(3)

In [ ]:
Vol(3.0)

In [ ]:
Vol("1.23")

In [ ]:
Vol(-2)

# Arithmetic

In [ ]:
Vol(3) + Vol(4)

In [ ]:
+(x::Vol, y::Vol) = Vol(x.value + y.value)

If we want to extend or override functions that already exit, we need to `import` them first.

In [ ]:
import Base: +

+(x::Vol, y::Vol) = Vol(x.value + y.value)

In [ ]:
Vol(3) + Vol(4)

In [ ]:
Vol(2) + Vol(8.3) # implicit conversion!

In [ ]:
methodswith(Vol)

In [ ]:
import Base: -, *

-(x::Vol, y::Vol) = Vol(x.value - y.value)
*(x::Vol, y::Vol) = Vol(x.value * y.value)

Now that we have addition defined for our volume type, some functions already **just work**.

In [ ]:
sum([Vol(3), Vol(4.8), Vol(1)])

In [ ]:
M = Vol.(rand(3,3))

In [ ]:
N = Vol.(rand(3,3))

In [ ]:
M + N

Whenever something doesn't work, we implement the necessary functions.

In [ ]:
sin(Vol(3))

In [ ]:
import Base: AbstractFloat
AbstractFloat(x::Vol{T}) where T = AbstractFloat(x.value)

In [ ]:
sin(Vol(3))

In [ ]:
sqrt(Vol(3))

If we really wanted to have `Vol{T}` objects behave like real numbers in all operations, we'd have to do a bit more work like specifying [promotion and conversion rules](https://docs.julialang.org/en/latest/manual/conversion-and-promotion/).

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our volume "wrapper" type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [ ]:
using BenchmarkTools

In [ ]:
operation(x) = x^2 + sqrt(x)

In [ ]:
x = rand(2,2)
@time operation.(x)

In [ ]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

In [ ]:
f()

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [ ]:
@benchmark operation.(x)

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [ ]:
@btime operation.(x);

However, we still have to take some care to avoid accessing global variables.

In [ ]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [ ]:
@btime sqrt(Vol(3));
@btime sqrt(3);

# Core messages of this Notebook

* There are `mutable struct`s and immutable `struct`s and immutability is not recursive.
* **Contructors** are functions that create objects. In an inner constructor we can use the function `new` to generate objects.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.

# Exercise: One-hot vector

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is useful in machine learning, as we'll see later.

It simply means that among a group of bits (all either 0 or 1) only one is hot (1) while all others are cold (0),

`v = [0, 0, 0, 0, 0, 1, 0, 0, 0]`

### Task

1. Think about what information an implementation of a one-hot vector actually has to store.
2. Define a `OneHot` type which represents a vector with only a single hot (i.e. `== 1`) bit.
3. Extend all the necessary `Base` functions such that the following computation works for a matrix `A` and a vector of `OneHot` vectors `vs` (i.e. `vs isa Vector{OneHot}`).

    ```julia
    function innersum(A, vs)
        t = zero(eltype(A)) # generic!
        for v in vs
            y = A*v
            for i in 1:length(vs[1])
                t += v[i] * y[i]
            end
        end
        return t
    end

    A = rand(3,3)
    vs = [rand(3) for _ = 1:10] # This should be replaced by a `Vector{OneHot}`

    innersum(A, vs)

    ```

4. Benchmark the speed of `innersum` when called with a `OneHot` vector or with a `Vector{Float64}`, respectively.
 * Do you observe a speed up?


5. Now, define a `OneHotVector` type which is identical to `OneHot` but is declared to be a subtype of `AbstractVector{Bool}` and extend only the functions `Base.getindex(v::OneHotVector, i::Int)` and `Base.size(v::OneHotVector)`.
 * Here, the function `size` should return a `Tuple{Int64}` indicating the length of the vector, i.e. `(3,)` for a one-hot vector of length 3 (see the [AbstractArray interface](https://docs.julialang.org/en/latest/manual/interfaces/#man-interface-array-1) for more information)
 

6. Try to create a single `OneHotVector` and try to run the `innersum` function using the new `OneHotVector` type.
 * What changes do you observe?
 * Do you have to implement any further methods?